In [1]:
import os

In [2]:
pwd

'c:\\Users\\Somridh Rudra\\Desktop\\ML\\complete_ml_proj\\online_fraud\\research'

In [3]:
os.chdir('../')

In [4]:
pwd

'c:\\Users\\Somridh Rudra\\Desktop\\ML\\complete_ml_proj\\online_fraud'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file:Path
    unzip_dir: Path

In [6]:
from house_price.constants import *
from house_price.utils.common import  read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH):
        

        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from house_price import logger
from house_price.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self,config):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [13]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2025-01-11 17:09:02,039: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-11 17:09:02,041: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-11 17:09:02,044: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-11 17:09:02,047: INFO: common: created directory at: artifacts]
[2025-01-11 17:09:02,050: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-11 17:09:03,158: INFO: 1501637349: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 49198
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "fbc236b47dda7431b1c9759dabdfa9b529b22b5d2770bd3b7cafbc82518c25b7"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 21BE:13E37D:316175:3F63E1:67825817
Accept-Ranges: bytes
Date: Sat